# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 3:  Búsqueda con Informacion (A*, Heuristicas)

Este laboratorio se centra en los métodos de búsqueda informada A*, y compararlos con la búsqueda ciega.

Se presenta un caso en el cual los algoritmos deberán buscar caminos en un laboritmo, encontrando la solución más óptima.


### Clase <b>Maze</b>
La clase Maze es representada por una matriz (grid) de espacios, los cuales pueden estar libres o bloqueados. Cada celda del grid puede tener uno de los siguientes carateres:

    '#': Indica una celda con obstaculo  (impasable)
    '~': Indica una celda con agua (pasable, con costo 5)
    ' ': Celda vacia (passable con costo 1)
    'S': Indicate la celda de inicio
    'E': Indicates la celda de salida

In [1]:
class Maze:

    def __init__(self, grid):
        """ Construye el maze a partir del grid pasado
            grid: debe ser una matriz (lista de listas), ejemplo [['#','S',' '],['#',' ','E']]  """
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        for i in range(self.numRows):
            for j in range(self.numCols):
                if len(grid[i]) != self.numCols:
                    raise "Grid no es Rectangular"
                if grid[i][j] == 'S':
                    self.startCell = (i,j)
                if grid[i][j] == 'E':
                    self.exitCell= (i,j)
        if self.exitCell == None:
            raise "No hay celda de Inicio"
        if self.startCell == None:
            raise "No hay celda de salida"

    def isPassable(self, row, col):
        """ Retorna true si la celda (row,col) es pasable  (' ' o '~') """
        return self.isWater(row, col) or self.isClear(row, col)

    def isWater(self, row, col):
        """ Retorna true si la celda (row,col) tiene agua  ('~') """
        return self.grid[row][col] == '~'

    def isClear(self, row, col):
        """ Retorna true si la celda (row,col) esta vacia  (' ') """
        return self.grid[row][col] == ' '

    def isBlocked(self, row,col):
        """ Retorna true si la celda (row,col) tiene obstaculo ('#') """
        return self.grid[row][col] == '#'

    def getNumRows(self):
        """ Retorna el numero de filas en el maze """
        return self.numRows

    def getNumCols(self):
        """ Retorna el numero de columnas en el maze """
        return self.numCols

    def getStartCell(self):
        """ Retorna la posicion (row,col) de la celda de inicio """
        return self.startCell

    def getExitCell(self):
        """ Retorna la posicion (row,col) de la celda de salida """
        return self.exitCell

    def __getAsciiString(self):
        """ Retorna el string de vizualizacion del maze """
        lines = []
        headerLine = ' ' + ('-' * (self.numCols)) + ' '
        lines.append(headerLine)
        for row in self.grid:
            rowLine = '|' + ''.join(row) + '|'
            lines.append(rowLine)
        lines.append(headerLine)
        return '\n'.join(lines)

    def __str__(self):
        return self.__getAsciiString()

### Funcion para cargar una laberinto de archivo de disco

In [2]:
def readMazeFromFile(file):
    """ Lee un archivo que contiene un laberinto y retorna una instancia de Maze con dicho laberinto"""
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines:
        grid.append(list(line))
    return Maze(grid)

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de búsqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de búsqueda.

In [3]:
class SearchProblem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con
        la accion action, asumiendo un costo c para llegar hasta state1.
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

###  <b> Clase MazeSearchProblem </b>  
Esta es una subclase de SearchProblem que implementa concretamente el problema de búsqueda en laberinto. El constructor recibe el laberinto en un objeto maze. Cada estado es codificado como una tupla (row,col) representando la posicion de una celda del grid. El estado inicial es la celda de inicio y el único estado objetivo es la celda de salida.
Se necesita completar Actions (acciones legales para un estado dado) y result (que hacen las acciones).

La clase MazeSearchProblem necesita ser completada (metodos actions y result). Deberá implementarse 2 heurísticas para A* (euclidean_dist:distancia en linea recta, y manhatan_dist:distancia Manhatan).

In [19]:
class MazeSearchProblem(SearchProblem):
    def __init__(self, maze):
        """El constructor recibe el maze"""
        self.maze = maze
        self.initial = maze.getStartCell()
        self.goal = maze.getExitCell()
        self.numNodesExpanded = 0
        self.expandedNodeSet = {}

    def __isValidState(self,state):
        """ Retorna true si el estado dado corresponde a una celda no bloqueada valida """
        row,col = state
        if row < 0 or row >= self.maze.getNumRows():
            return False
        if col < 0 or col >= self.maze.getNumCols():
            return False
        return not self.maze.isBlocked(row,col)

    def actions(self, state):
        """Retorna las acciones legales desde la celda actual """
        row,col = state
        acciones = []
        if self.__isValidState((row-1, col)):
            acciones.append('N')
        ############################################################
        ##COMPLETAR
        if self.__isValidState((row+1, col)):
            acciones.append('S')
        if self.__isValidState((row, col+1)):
            acciones.append('E')
        if self.__isValidState((row, col-1)):
            acciones.append('W')

        ############################################################
        return acciones

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde la celda actual.
        La accion debe ser alguna de self.actions(state)"""
        row,col = state
        newState = ()
        if action == 'N':
            newState = (row-1, col)
        ############################################################
        ##COMPLETAR
        if action == 'S':
            newState = (row+1, col)
        if action == 'E':
            newState = (row, col+1)
        if action == 'W':
            newState = (row, col-1)
        ############################################################
        return newState

    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        return (self.goal == state)

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action
        El costo del camino para llegar a state1 es c. El costo de la accion sale de self.maze """
        row, col = state2
        if self.maze.isClear(row, col):
            actionCost = 1
        elif self.maze.isWater(row, col):
            actionCost = 5
        elif state2 == self.maze.getStartCell() or state2 == self.maze.getExitCell():
            actionCost = 1
        else:
            raise "El costo de una celda bloqueada no esta definido"
        return c + actionCost

### Clase <b>Node</b>

Estructura de datos para almacenar la informacion de un nodo en un <b>arbol de búsqueda</b>. Contiene información del nodo padre y el estado que representa el nodo. Tambien incluye la accion que nos llevo al presente nodo y el costo total del camino desde el nodo raiz hasta este nodo.

In [5]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de búsqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __lt__(self, node):
        return self.state < node.state

    def __eq__(self, other):
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __hash__(self):
        return hash(self.state)

### <b> Frontera tipo cola FIFO (first-in first out) para BFS</b>

In [6]:
from collections import deque

class FIFOQueue(deque):
    """Una cola First-In-First-Out"""
    def pop(self):
        return self.popleft()

### <b> Frontera tipo cola de prioridad ordenada por una funcion de costo (para best_first_graph_search y A*)</b>

In [7]:
import heapq
class FrontierPQ:
    "Una Frontera ordenada por una funcion de costo (Priority Queue)"

    def __init__(self, initial, costfn=lambda node: node.path_cost):
        "Inicializa la Frontera con un nodo inicial y una funcion de costo especificada (por defecto es el costo de camino)."
        self.heap   = []
        self.states = {}
        self.costfn = costfn
        self.add(initial)

    def add(self, node):
        "Agrega un nodo a la frontera."
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        self.states[node.state] = node

    def pop(self):
        "Remueve y retorna el nodo con minimo costo."
        (cost, node) = heapq.heappop(self.heap)
        self.states.pop(node.state, None) # remove state
        return node

    def replace(self, node):
        "node reemplaza al nodo de la Fontera que tiene el mismo estado que node."
        if node.state not in self:
            raise ValueError('{} no tiene nada que reemplazar'.format(node.state))
        for (i, (cost, old_node)) in enumerate(self.heap):
            if old_node.state == node.state:
                self.heap[i] = (self.costfn(node), node)
                heapq._siftdown(self.heap, 0, i)
                return

    def __contains__(self, state): return state in self.states

    def __len__(self): return len(self.heap)

### <b>Algoritmo general de búsqueda con memoria de nodos expandidos (Graph Search)</b>

Algoritmo de general de búsqueda ciega con memoria de estados visitados. El argumento frontier debe ser una cola vacia. Si la frontera es tipo FIFO hace búsqueda en amplitud (BFS), si la frontera es una pila hará búsqueda en profundidad (DFS)

In [8]:
def graph_search(problem, frontier):
    frontier.append(Node(problem.initial))
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la búsqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)

        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

### <b> Algoritmo Best-First-Graph-Search </b>
Algoritmo general de búsqueda con información. La frontera es una cola de prioridad ordenada por la funcion de evaluacion f

In [9]:
def best_first_graph_search(problem, f):
    """Busca el objetivo expandiendo el nodo de la frontera con el menor valor de la funcion f. Memoriza estados visitados
    Antes de llamar a este algoritmo hay que especificar La funcion f(node). Si f es node.depth tenemos Búsqueda en Amplitud;
    si f es node.path_cost tenemos Búsqueda  de Costo Uniforme. Si f es una heurística tenemos Búsqueda Voraz;
    Si f es node.path_cost + heuristica(node) tenemos A* """

    frontier = FrontierPQ( Node(problem.initial), f )  # frontera tipo cola de prioridad ordenada por f
    explored = set()     # memoria de estados visitados
    visited_nodes = []   # almacena nodos visitados durante la búsqueda
    while frontier:
        node = frontier.pop()
        visited_nodes.append(node)
        if problem.goal_test(node.state):
            return node, visited_nodes
        explored.add(node.state)
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            if child.state not in explored and child.state not in frontier:
                frontier.add(child)
            elif child.state in frontier:
                incumbent = frontier.states[child.state]
                if f(child) < f(incumbent):
                    frontier.replace(child)

### <b> Algoritmo A* </b>
A* es un caso especial de best_first_graph_search con f = path_cost + heuristic

In [10]:
def astar_search(problem, heuristic):
    f = lambda node: node.path_cost + heuristic(node, problem)
    return best_first_graph_search(problem, f)

### <b> Heurísticas para A* </b>
Se debe implementar las heurísticas abajo para A*

In [11]:
import math
def euclidean_dist(node, problem):
    "Distancia en linea recta desde la celda de node hasta la celda Objetivo (problem.goal)"
    delta_x = node.state[0] - problem.goal[0]  # distancia desde node al objetivo en el eje x
    delta_y = node.state[1] - problem.goal[1]  # distancia desde node al objetivo en el eje y
    return math.sqrt( math.pow(delta_x, 2) + math.pow(delta_y, 2) )

def manhatan_dist(node, problem):
    "Distancia Manhatan desde la celda de node hasta la celda Objetivo (problem.goal)"
    delta_x = node.state[0] - problem.goal[0]
    delta_y = node.state[1] - problem.goal[1]
    "Distancia Manhatan. La suma de las distancias a lo largo de cualquiera de sus dimensiones coordenadas."
    return abs(delta_x) + abs(delta_y)

def chebyshev_dist(node, problem):
    "Distancia Chebyshev desde la celda de node hasta la celda Objetivo (problem.goal)"
    delta_x = node.state[0] - problem.goal[0]
    delta_y = node.state[1] - problem.goal[1]
    "Distancia Chebyshev. La mayor de las distancias a lo largo de cualquiera de sus dimensiones coordenadas."
    return  max(abs(delta_x),abs(delta_y))

def nullheuristic(node, problem):   # heurística nula (A* se convierte en búsqueda de costo uniforme)
    return 0

### <b> Funcion para mostrar los resultados </b>

In [12]:
def displayResults(maze, visitedNodes, solutionNodes):
    """ Muestra los resultados de búsqueda en el maze.   """
    grid_copy = []
    for row in maze.grid:
        grid_copy.append([x for x in row])
    for node in visitedNodes:
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'o'
    for node in solutionNodes:
        row,col = node.state
        ch = maze.grid[row][col]
        if ch != 'S' and ch != 'E': grid_copy[row][col] = 'x'
    maze_copy = Maze(grid_copy)
    print (maze_copy)
    print ("x - celdas en la solucion")
    print ("o - celdas visitadas durante la búsqueda")
    print ("-------------------------------")

## <b> Experimentación con los algoritmos de Búsqueda</b>


In [21]:
""" Carga un laberinto de archivo en disco e instancia el problema de búsqueda.   """
maze = readMazeFromFile('maze.txt')
p = MazeSearchProblem(maze)
print(maze)

 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~          ~~~~|
|~~~~            ~~~|
|########   ~~~~~~~~|
|             ######|
|      S      #~~~~~|
|###### # #####~~~~~|
|     # #      ~~~~~|
| # # ###### ####~~~|
| #          #~~~~~~|
|     ### ####~~~~~~|
|                  E|
 ------------------- 


### Búsqueda en amplitud (BFS)

In [22]:
nsol, visited_nodes = graph_search(p, FIFOQueue())
print('Solucion BFS: {}. Nodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion BFS: ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'S', 'S', 'E', 'E']. Nodos visitados=180. Costo Solucion = 42
 ------------------- 
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|########ooooooooooo|
|ooooooooooooo######|
|ooooooSxxoooo#ooooo|
|######o#x#####ooooo|
|    o#o#xxxxxxxxxoo|
| # #o######o####xoo|
| #oooooooooo#oooxoo|
|   oo###o####~ooxoo|
|    ooooooooo  oxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### Búsqueda en profundidad (DFS)

In [23]:
nsol, visited_nodes = graph_search(p, [])
print('Solucion DFS: {}. \nNodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion DFS: ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E']. 
Nodos visitados=174. Costo Solucion = 80
 ------------------- 
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|ooooooooooooooooooo|
|########ooooooooooo|
|oooooo   oooo######|
|ooooooSxxoooo#~~~~~|
|###### #x#####~~~~~|
|ooooo# #xxxxxxxxxxx|
|o#o#o###### ####~~x|
|o#oooooooooo#xxxxxx|
|ooooo###o####x~~~~~|
|oooooooooooooxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### Búsqueda A* con heurística nula (UCS)

In [24]:
nsol, visited_nodes = astar_search(p, nullheuristic)
print('Solucion A* y heuristica nula (UCS): {}. \nNodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica nula (UCS): ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']. 
Nodos visitados=152. Costo Solucion = 24
 ------------------- 
|~~~~ooooooooooo~~~~|
|~~~ooooooooooooo~~~|
|~~ooooooooooooooo~~|
|~ooooooooooooooooo~|
|########ooooooooo~~|
|ooooooooooooo######|
|ooooooSxxoooo#o~~~~|
|######o#x#####oo~~~|
|ooooo#o#xxxxooooo~~|
|o#o#o######x####~~~|
|o#ooooooxxxx#~~~~~~|
|ooooo###x####o~~~~~|
|ooooooooxxxxxxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### Búsqueda A* con heurística euclidean_dist

In [25]:
nsol, visited_nodes = astar_search(p, euclidean_dist)
print('Solucion A* y heuristica euclidean_dist: {}. \nNodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica euclidean_dist: ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']. 
Nodos visitados=76. Costo Solucion = 24
 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~ ooooooooo~~~~|
|~~~~ ooooooooooo~~~|
|########ooooo~~~~~~|
|  ooooooooooo######|
| oooooSxxoooo#~~~~~|
|######o#x#####~~~~~|
|     #o#xxxxoooo~~~|
| # # ######x####~~~|
| #      xxxx#~~~~~~|
|     ###x####~~~~~~|
|        xxxxxxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### Búsqueda A* con heurística manhatan_dist

In [26]:
nsol, visited_nodes = astar_search(p, manhatan_dist)
print('Solucion A* y heuristica manhatan_dist: {}. \nNodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica manhatan_dist: ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']. 
Nodos visitados=57. Costo Solucion = 24
 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~~~~~~~~~~~~~|
|~~~~~          ~~~~|
|~~~~    oooooooo~~~|
|########ooo~~~~~~~~|
|    ooooooooo######|
|   oooSxxoooo#~~~~~|
|######o#x#####~~~~~|
|     #o#xxxxooo~~~~|
| # # ######x####~~~|
| #      xxxx#~~~~~~|
|     ###x####~~~~~~|
|        xxxxxxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


### Búsqueda A* con heurística chebyshev_dist

In [27]:
nsol, visited_nodes = astar_search(p, chebyshev_dist)
print('Solucion A* y heuristica manhatan_dist: {}. \nNodos visitados={}. Costo Solucion = {}'.format(nsol.solution(), len(visited_nodes),nsol.path_cost))
displayResults(maze, visited_nodes, nsol.path())

Solucion A* y heuristica manhatan_dist: ['E', 'E', 'S', 'S', 'E', 'E', 'E', 'S', 'S', 'W', 'W', 'W', 'S', 'S', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E']. 
Nodos visitados=88. Costo Solucion = 24
 ------------------- 
|~~~~~~~~~~~~~~~~~~~|
|~~~~~~~oooo~~~~~~~~|
|~~~~~oooooooooo~~~~|
|~~~~oooooooooooo~~~|
|########ooooooo~~~~|
| oooooooooooo######|
|ooooooSxxoooo#~~~~~|
|######o#x#####o~~~~|
|     #o#xxxxoooo~~~|
| # # ######x####~~~|
| #     oxxxx#~~~~~~|
|     ###x####~~~~~~|
|        xxxxxxxxxxE|
 ------------------- 
x - celdas en la solucion
o - celdas visitadas durante la búsqueda
-------------------------------


#Preguntas

1) Complete el código necesario. <b>(4 pts)</b>

2) En el anterior laboratorio, BFS garantizaba soluciones óptimas. ¿En cuanto a los resultados obtenidos, los algoritmos BFS o DFS encuentran soluciones optimas? ¿Por qué? <b>(3 pts)</b>

Solo 'bfs' encuentra soluciones óptimas, ya que en busca por amplitud encontrando la solución en el nivel más bajo pero con la desventaja de pasar por demasiados nodos y ocupar mucho espacio. En el caso de 'dfs' no encuentra una solución óptima ya que lo que hace es buscar por profundidad encontrando muchas veces una solución larga, habiendo muchos casos donde exista una solución más corta y óptima.

3) ¿En cuanto a los resultados obtenidos, las soluciones obtenidas por A* serán siempre óptimas? ¿Por qué? <b>(3 pts)</b>

Solamente se podrá obtener soluciones óptimas si se usa una heurística admisible y consistente, junto con su costo de moivimiento, ya que usando esta información se creará nodos que estén más proximos al objetivo y con el costo mínimo.

4) ¿Puedes pensar en situaciones del mundo real en las que este algoritmo A* modificado con penalización según el tipo de camino podría ser útil? Plantea un ejemplo. <b>(3 pts)</b>

Exise muhcos ejemplos en la vida real, puede ser simplemente con un robotde entrega de comida, donde no debe pasar por agua, encuentre el camino más óptimo sin tocar esta, o puede ser con el simple hecho de gasto de combustible y reducir el costo de este al máximo. También en la planificación de viaje de los gps, para evitar el trafico y reducir el tiempo de espera para llegar de un sitio a otro

5) Explique el concepto de dominancia entre heurísticas <b>(2 pts)</b>

Una heurística es más dominante cuadno es mayor a otra, esto provoca que la heruistica dominante expanda resultados más proximos al objetivo a comparación de otras heurísticas. En el caso de manhattan, se suma los valores absolutos de esta y por eso es la heurística dominante.

6) ¿Cual es la heuristica más dominante? ¿Por qué? <b>(2 pts)</b>

La heurística dominante es la que tiene datos mayores a otras. La heuristica dominante sería la Manhattan ya que esta suma los valores absolutos de las coordenadas a comparacion de euclidiana donde hay raiz cuadrada y Chebyshev donde solo se sacala mayor coordenada de distancia.

Manhattan = abs(distCordX) + abs(distCordY)

Euclidiana =  sqrt(distCordX^2 + distCordY^2 )
Chebyshev = max(distCordX, distCordY)

Manhattan > euclidiana >= chebysev

7) Basado en la teoría y en los ejemplos vistos, ¿cuál heuristica tiene un menor costo temporal y de memoria? ¿Por qué? <b>(3 pts)</b>

Viendo los ejemplos y usando la teoría de b* y heurística dominante podemos notar que la heurística que tiene menor costo temporal y de memoria es la Manhattan ya que visita menos nodos que otras heurísticas (57), Y obtiene una solución óptima.